In [9]:
from huggingface_hub import notebook_login
notebook_login()

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [49]:
from transformers import AutoModelForMaskedLM, AutoModel, AutoTokenizer, GPT2Tokenizer, AutoModelForCausalLM
import pandas as pd
import numpy as np
import re
#original Version: 4.24.4
from evaluate import load
bertscore = load("bertscore")

## Generating Agent Predictions 

In [59]:
def generate(context, model, tokenizer):
  # create prompt
  prompt = context + " agent: "
  # print("prompt: ", prompt)

  # tokenize input (string -> tensor)
  input_ids = tokenizer(prompt, return_tensors='pt').input_ids

  # also save to csv to do a sanity check
  output_ids = model.generate(input_ids, max_length=800, eos_token_id=tokenizer.encode("customer: "))
  
  # decode output (tensor -> string)
  pred_response = tokenizer.batch_decode(output_ids, skip_special_tokens = True)


  # slice the output: output_ids = outputs[:, input_ids.shape[-1]:]
  output_string = pred_response[0].strip()
  prefix = context.strip() + " agent: "

  #regex
  output_string = re.sub("\.+", "", output_string)
  prefix = re.sub("\.+", "", prefix)
  prefix = re.sub("  ", " ", prefix)
  output_string = re.sub("  ", " ", output_string)

  suffix1 = "customer:"
  suffix2 = "agent:"
  # check for both suffix "customer: and agent:" -> remove for consistnecy 
  # og_output.append(output_string.removeprefix(prefix))
  sliced_output = output_string.removeprefix(prefix.strip())
  print(sliced_output)
  if sliced_output.endswith(suffix1): 
    sliced_output = sliced_output.rstrip(suffix1)
  if sliced_output.endswith(suffix2):
      sliced_output = sliced_output.rstrip(suffix2)
  
  print(sliced_output)


  return sliced_output

In [56]:
import re
"customer: Hi agent: Good afternoon, what can I do for you? customer: Hi would like to dispute a bill I got charged twice Hello? agent: Understood, could you give me your full name, ID and order number please customer: Joseph Banter I don't have an order # Hello? agent: let me review a solution, sorry for the wait customer: No problem agent: could you give me your email please action: Identity verification in progess action: Details of Bills have been entered customer: I'm next josephb4@emailcom action: Purchase validation in progress agent: When did you make your purchase? are you there friend? action: Querying the system for an answer customer: Last week?? agent: Friend it turns out that if there was an error in your bill from us customer: Thank you Please fix that's it action: Order has been updated with give credit agent: Sorry the inconvenience and I will make an arrangement to solve your problem customer: Thank you that's l needed done agent: I will need your username, email address and order number please customer:"
"customer: Hi agent: Good afternoon, what can I do for you? customer: Hi would like to dispute a bill I got charged twice Hello? agent: Understood, could you give me your full name, ID and order number please customer: Joseph Banter I don't have an order # Hello? agent: let me review a solution, sorry for the wait customer: No problem agent: could you give me your email please action: Identity verification in progess action: Details of Bills have been entered customer: I'm next josephb4@emailcom action: Purchase validation in progress agent: When did you make your purchase? are you there friend? action: Querying the system for an answer customer: Last week?? agent: Friend it turns out that if there was an error in your bill from us customer: Thank you Please fix that's it action: Order has been updated with give credit agent: Sorry the inconvenience and I will make an arrangement to solve your problem customer: Thank you that's l needed done agent: I will need your username, email address and order number please "


"customer: Hi agent: Good afternoon, what can I do for you? customer: Hi would like to dispute a bill I got charged twice Hello? agent: Understood, could you give me your full name, ID and order number please customer: Joseph Banter I don't have an order # Hello? agent: let me review a solution, sorry for the wait customer: No problem agent: could you give me your email please action: Identity verification in progess action: Details of Bills have been entered customer: I'm next josephb4@emailcom action: Purchase validation in progress agent: When did you make your purchase? are you there friend? action: Querying the system for an answer customer: Last week?? agent: Friend it turns out that if there was an error in your bill from us customer: Thank you Please fix that's it action: Order has been updated with give credit agent: Sorry the inconvenience and I will make an arrangement to solve your problem customer: Thank you that's l needed done agent: I will need your username, email addr

In [79]:
model = AutoModelForCausalLM.from_pretrained("../finetuning/gpt2-medium-finetuned-noaction-ABCD").eval()
tokenizer = AutoTokenizer.from_pretrained("gpt2")

In [81]:
# df = pd.read_csv("evaluation_csv_v1.csv")
df = pd.read_excel("evaluation_concat_noaction.xlsx")

# need to do subsample
df_subsample = df.sample(n=1000, random_state=1) 
output_df = pd.DataFrame(columns=["context", "true_agent_response", "predicted_response", 'BERT_precision', 'BERT_recall', "BERT_f1"])
all_pred_responses, all_true_responses = [], []

for index, row in df_subsample.iterrows():
    pred_response = generate(row["context"], model, tokenizer)
    all_pred_responses.append(pred_response)
    all_true_responses.append(row["true_agent_response"]) 
    similarity_score = bertscore.compute(predictions=[pred_response], references=[row["true_agent_response"]], lang="en", rescale_with_baseline = True)
    new_row = {"context": row["context"], "true_agent_response":row["true_agent_response"], "predicted_response": pred_response, 'BERT_precision':similarity_score["precision"][0], "BERT_recall": similarity_score["recall"][0], "BERT_f1": similarity_score["f1"][0]}
    output_df = pd.concat([output_df, pd.DataFrame([new_row])], ignore_index=True)

output_df.to_excel("gpt2medium-noaction_concat.xlsx")

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:23144 for open-end generation.


 I am sorry to hear that
 I am sorry to hear that


/var/folders/ml/t6mthscj76j1mqyxmp4kk9880000gn/T/ipykernel_75896/2794441537.py:15: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  output_df = pd.concat([output_df, pd.DataFrame([new_row])], ignore_index=True)
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:23144 for open-end generation.


customer: Hi agent: Good afternoon, what can I do for you? customer: Hi would like to dispute a bill I got charged twice Hello? agent: Understood, could you give me your full name, ID and order number please customer: Joseph Banter I don't have an order # Hello? agent: let me review a solution, sorry for the wait customer: No problem agent: could you give me your email please customer: I'm next josephb4@emailcom agent: When did you make your purchase? are you there friend? customer: Last week?? agent: Friend it turns out that if there was an error in your bill from us customer: Thank you Please fix that's it agent: Sorry the inconvenience and I will make an arrangement to solve your problem customer: Thank you that's l needed done agent: I have sent a link to your email so you can check the status of your subscription customer:
customer: Hi agent: Good afternoon, what can I do for you? customer: Hi would like to dispute a bill I got charged twice Hello? agent: Understood, could you giv

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:23144 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:23144 for open-end generation.


...

 What is your question? customer:
 What is your question? 


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:23144 for open-end generation.


 Is there anything else I can help you with? customer:
 Is there anything else I can help you with? 


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:23144 for open-end generation.


 thanks! agent:
 thanks! 
 And what was the brand and type of item you ordered? customer:
 And what was the brand and type of item you ordered? 


## Evaluation Using BERT

In [88]:
from evaluate import load
bertscore = load("bertscore")
predictions = ["hello there", "general kenobi"]
references = ["hello there", "general kenobi"]
results = bertscore.compute(predictions=predictions, references=references, lang="en")

In [89]:
results

{'precision': [0.9999998807907104, 1.0000001192092896],
 'recall': [0.9999998807907104, 1.0000001192092896],
 'f1': [0.9999998807907104, 1.0000001192092896],
 'hashcode': 'roberta-large_L17_no-idf_version=0.3.12(hug_trans=4.28.0)'}